In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd

import contextily as ctx

import xml.etree.ElementTree as ET

In [48]:
tree = ET.parse('data/ride.tcx')
root = tree.getroot()

In [49]:
data = []
for trackpoint in root.findall('.//{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Trackpoint'):
    time = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Time').text
    lat = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position/{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LatitudeDegrees').text
    lon = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position/{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LongitudeDegrees').text
    altitude = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}AltitudeMeters').text
    distance = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}DistanceMeters').text
    data.append([time, float(lat), float(lon), float(altitude), float(distance)])

df = pd.DataFrame(data, columns=['Time', 'Latitude', 'Longitude', 'AltitudeMeters', 'DistanceMeters'])


In [50]:
df.head()

,Time,Latitude,Longitude,AltitudeMeters,DistanceMeters
0,2023-09-22T22:41:40+00:00,37.998980,-1.13315,46.02,0.000000
1,2023-09-22T22:41:49+00:00,37.998233,-1.13288,45.98,86.408879
2,2023-09-22T22:41:59+00:00,37.997487,-1.13261,45.68,172.817758
3,2023-09-22T22:42:09+00:00,37.996740,-1.13234,43.00,259.226638
4,2023-09-22T22:42:19+00:00,37.996590,-1.13301,43.69,320.315240


# Calculating the distance

In [51]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

In [52]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

In [53]:
total_distance = 0
for i in range(1, len(df)):
    lat1, lon1 = df['Latitude'][i - 1], df['Longitude'][i - 1]
    lat2, lon2 = df['Latitude'][i], df['Longitude'][i]
    
    seg_distance = haversine(lat1, lon1, lat2, lon2)
    total_distance += seg_distance
    
f"Total Distance: {total_distance:.2f} km"

'Total Distance: 84.79 km'

# Calculating the duration

In [54]:
df.head()

,Time,Latitude,Longitude,AltitudeMeters,DistanceMeters
0,2023-09-22T22:41:40+00:00,37.998980,-1.13315,46.02,0.000000
1,2023-09-22T22:41:49+00:00,37.998233,-1.13288,45.98,86.408879
2,2023-09-22T22:41:59+00:00,37.997487,-1.13261,45.68,172.817758
3,2023-09-22T22:42:09+00:00,37.996740,-1.13234,43.00,259.226638
4,2023-09-22T22:42:19+00:00,37.996590,-1.13301,43.69,320.315240


In [55]:
df['Time'] = pd.to_datetime(df['Time'])

In [56]:
df = df.sort_values(by='Time')
df['TimeDiff'] = df['Time'].diff()
total_moving_time = df['TimeDiff'].sum()

display(f"Total Moving Time: {total_moving_time}")


'Total Moving Time: 0 days 04:04:31'

# Calculate elevation loss/gain

In [57]:
df.head()

,Time,Latitude,Longitude,AltitudeMeters,DistanceMeters,TimeDiff
0,2023-09-22 22:41:40+00:00,37.998980,-1.13315,46.02,0.000000,NaT
1,2023-09-22 22:41:49+00:00,37.998233,-1.13288,45.98,86.408879,0 days 00:00:09
2,2023-09-22 22:41:59+00:00,37.997487,-1.13261,45.68,172.817758,0 days 00:00:10
3,2023-09-22 22:42:09+00:00,37.996740,-1.13234,43.00,259.226638,0 days 00:00:10
4,2023-09-22 22:42:19+00:00,37.996590,-1.13301,43.69,320.315240,0 days 00:00:10


In [58]:
df['AltitudeChange'] = df['AltitudeMeters'].diff()
df.head()

,Time,Latitude,Longitude,AltitudeMeters,DistanceMeters,TimeDiff,AltitudeChange
0,2023-09-22 22:41:40+00:00,37.998980,-1.13315,46.02,0.000000,NaT,NaN
1,2023-09-22 22:41:49+00:00,37.998233,-1.13288,45.98,86.408879,0 days 00:00:09,-0.04
2,2023-09-22 22:41:59+00:00,37.997487,-1.13261,45.68,172.817758,0 days 00:00:10,-0.30
3,2023-09-22 22:42:09+00:00,37.996740,-1.13234,43.00,259.226638,0 days 00:00:10,-2.68
4,2023-09-22 22:42:19+00:00,37.996590,-1.13301,43.69,320.315240,0 days 00:00:10,0.69


In [68]:
total_ascent = df['AltitudeChange'][df['AltitudeChange'] > 0].sum()
total_descent = df['AltitudeChange'][df['AltitudeChange'] < 0].sum()
total_change = df['AltitudeChange'].sum()
lowest = df['AltitudeMeters'].min()
highest = df['AltitudeMeters'].max()

2.4899999999999998